# Nahuatl Notebook for the WHP_EarlyNahuatl_Dataset

This notebook processes Nahuatl dictionary data, analyzing HTML tags, repairing malformed tags, and extracting citations and cross-references. This is a merged version of Todd's version and I where there is a SQLite-based data management approach.

## Setup and Imports

In [18]:
import pandas as pd
import re
import os
import numpy as np
import hashlib
import glob
import csv
import sqlite3
from collections import defaultdict, Counter
from typing import List, Dict, Tuple, Optional, Union
from inscriptis import get_text
from bs4 import BeautifulSoup
from pathlib import Path
from datetime import datetime

In [19]:
# Create working directory
os.makedirs('working_files', exist_ok=True)

# load in the SQLite database holding the WHP Dataset
conn = sqlite3.connect('../../data/sqLiteDb/Whp_Raw_Dataset.db')
table_name = "WHP_EarlyNahuatl_Data"

tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(tables_query, conn)
tables


# If there's issues check the following
# Possible solutions:
# 1. Ensure the db file is in the correct directory
# 2. Check the exact filename
# 3. Verify the file extension

,name
0,WHP_EarlyNahuatl_Data
1,WHP_EarlyNahuatl_Deduplicated


In [20]:
def save_to_excel(data_dict: Dict[str, pd.DataFrame], filename: str, directory: str = 'working_files'):
    """Save multiple DataFrames as sheets in an Excel file"""
    filepath = os.path.join(directory, filename)
    with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
        for sheet_name, df in data_dict.items():
            # Truncate sheet name if too long (Excel limit is 31 characters)
            clean_sheet_name = sheet_name[:31] if len(sheet_name) > 31 else sheet_name
            df.to_excel(writer, sheet_name=clean_sheet_name, index=False)
    print(f"Saved to: {filepath}")

In [21]:
def save_dataframe(df: pd.DataFrame, filename: str, directory: str = 'working_files'):
    """Save a single DataFrame to CSV"""
    filepath = os.path.join(directory, filename)
    df.to_csv(filepath, index=False)
    print(f"Saved to: {filepath}")

In [22]:
def save_to_sqlite(df: pd.DataFrame, table_name: str, conn: sqlite3.Connection, if_exists: str = 'replace'):
    """Save DataFrame to SQLite table"""
    df.to_sql(table_name, conn, if_exists=if_exists, index=False)
    print(f"Saved to SQLite table: {table_name}")

## Step 1: Import Data and Create Working Copy

In [23]:
def load_data_from_csv(filename: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Load data and create a working copy"""
    print(f"Loading data from: {filename}")

    # Read the original data
    original_df = pd.read_csv(filename)

    # Create working copy
    working_df = original_df.copy()

    print(f"Data loaded successfully:")
    print(f"- Shape: {original_df.shape}")
    print(f"- Columns: {list(original_df.columns)}")

    return original_df, working_df

In [24]:
def load_data_from_sqlite(db_path: str, table_name: str = "WHP_EarlyNahuatl_Data") -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Load data from SQLite and create a working copy"""
    print(f"Loading data from: {db_path}")
    
    conn = sqlite3.connect(db_path)
    original_df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    working_df = original_df.copy()
    
    print(f"Data loaded successfully:")
    print(f"- Shape: {original_df.shape}")
    print(f"- Columns: {list(original_df.columns)}")
    
    # Don't close connection yet - return it for later use
    return original_df, working_df, conn

In [27]:
# Load your data

original_df = pd.read_sql("SELECT * FROM WHP_EarlyNahuatl_Data", conn)
df = original_df.copy()

query = "SELECT * FROM WHP_EarlyNahuatl_Data LIMIT 3;"
whp_dataset = pd.read_sql(query, conn)
display(whp_dataset)

cursor = conn.execute(f"PRAGMA table_info({table_name})")
columns_info = cursor.fetchall()
column_names = [col[1] for col in columns_info]

print(column_names)

,Ref,Headword,Orthographic Variants,Principal English Translation,Attestations from sources in English,Attestations from sources in Spanish,Alonso de Molina,Frances Karttunen,Horacio Carochi / English,Andrés de Olmos,Lockhart’s Nahuatl as Written,themes,Spanish Loanword
0,WHP-171879,acazomo.,"accaçomo, acaçomo",<p>perhaps not (adverb) (see Molina)</p>,<p>acaçomo iuhqui yez yn anoço yuhquiez = whet...,None,<p>Acaçomo. quiça no. Aduerbio.<br /> <bibl> A...,<p>AHCAZOMŌ perhaps not / quizá no (M). In on...,<p>àcaçomō = perhaps not<br /> <bibl>Horacio C...,None,None,None,No
1,WHP-171881,ayac.,aiaac,"<p>no one; nobody; or, for someone to be absen...",<p>aiaac mic in mexica = None of the Mexica di...,<p>ayac guincuiliz = no se la quite nadie (Tla...,"<p>Ayac. ninguno, o nadie o estar alguno ausen...","<p>AYĀC no one / ninguno, o nadie (M) See AH-,...","<p>ayāc = no one<br /> <bibl>Horacio Carochi, ...",None,"<p>no one; nobody; or, for someone to be absen...",None,No
2,WHP-171882,acan.,None,"<p>nowhere, no place (see Molina, Karttunen, L...",None,None,<p>acan. en ninguna parte o lugar. aduerbio.<b...,<p>AHCĀN nowhere / en ninguna parte o lugar (M...,"<p>àcān = nowhere<br /> <bibl>Horacio Carochi,...","<p>en ningun lugar, por, de, etc.<br /> <bibl>...",<p>ahcān = (particle) nowhere<br /> <bibl>Jame...,"Cardinal Directions, Cosmos",No


['Ref', 'Headword', 'Orthographic Variants', 'Principal English Translation', 'Attestations from sources in English', 'Attestations from sources in Spanish', 'Alonso de Molina', 'Frances Karttunen', 'Horacio Carochi / English', 'Andrés de Olmos', 'Lockhart’s Nahuatl as Written', 'themes', 'Spanish Loanword']


## Step 2: Save Intermediate Stages

In [16]:
def save_intermediate_stage(df: pd.DataFrame, stage_name: str):
    """Save intermediate processing stage"""
    filename = f"{stage_name}_stage.csv"
    save_dataframe(df, filename)
    return df

def save_intermediate_stage_sqlite(df: pd.DataFrame, stage_name: str, conn: sqlite3.Connection):
    """Save intermediate processing stage to SQLite"""
    table_name = f"{stage_name}_stage"
    save_to_sqlite(df, table_name, conn)
    return df

In [29]:
# Save initial stage
save_intermediate_stage_sqlite(df, "01_initial", conn)

Saved to SQLite table: 01_initial_stage


,Ref,Headword,Orthographic Variants,Principal English Translation,Attestations from sources in English,Attestations from sources in Spanish,Alonso de Molina,Frances Karttunen,Horacio Carochi / English,Andrés de Olmos,Lockhart’s Nahuatl as Written,themes,Spanish Loanword
0,WHP-171879,acazomo.,"accaçomo, acaçomo",<p>perhaps not (adverb) (see Molina)</p>,<p>acaçomo iuhqui yez yn anoço yuhquiez = whet...,None,<p>Acaçomo. quiça no. Aduerbio.<br /> <bibl> A...,<p>AHCAZOMŌ perhaps not / quizá no (M). In on...,<p>àcaçomō = perhaps not<br /> <bibl>Horacio C...,None,None,None,No
1,WHP-171881,ayac.,aiaac,"<p>no one; nobody; or, for someone to be absen...",<p>aiaac mic in mexica = None of the Mexica di...,<p>ayac guincuiliz = no se la quite nadie (Tla...,"<p>Ayac. ninguno, o nadie o estar alguno ausen...","<p>AYĀC no one / ninguno, o nadie (M) See AH-,...","<p>ayāc = no one<br /> <bibl>Horacio Carochi, ...",None,"<p>no one; nobody; or, for someone to be absen...",None,No
2,WHP-171882,acan.,None,"<p>nowhere, no place (see Molina, Karttunen, L...",None,None,<p>acan. en ninguna parte o lugar. aduerbio.<b...,<p>AHCĀN nowhere / en ninguna parte o lugar (M...,"<p>àcān = nowhere<br /> <bibl>Horacio Carochi,...","<p>en ningun lugar, por, de, etc.<br /> <bibl>...",<p>ahcān = (particle) nowhere<br /> <bibl>Jame...,"Cardinal Directions, Cosmos",No
3,WHP-171883,acampa.,None,<p>from nowhere; in no way; neither from one p...,None,None,"<p>Acampa. de ninguna parte, o ni a vna parte ...",None,<p>àcāmpa = nowhere<br /> <bibl>Horacio Caroch...,None,None,"Cardinal Directions, Cosmos",No
4,WHP-171884,acatto.,"acattopa, yacatopa",<p>first (see Karttunen)</p>,"<p>YACATTO, YACATTOPA = first / primero<br /> ...",None,None,None,None,None,None,"Numbers, Math",No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31801,WHP-211066,Teyahualco.,None,"<p>a place name; e.g., the town of Santiago Te...",<p>The place name components: <em>te</em>- sto...,None,None,None,None,None,None,None,No
31802,WHP-211067,chinancocol.,None,<p>a type of agricultural field--wavy? with ri...,None,None,None,None,None,None,None,None,No
31803,WHP-211068,poyahuallotl.,None,<p>a feather that covers the bird's tail feath...,None,"<p>""La pluma que tienen las aues cerca de la c...",None,None,None,None,None,None,No
31804,WHP-211069,Cuauhquiyahuacatl.,None,<p>a title given to a principal for distinguis...,None,"<p>""Mecatzin. Uno de los valerosos ""soldados c...",None,None,None,None,None,None,No


## Step 3: HTML Tag Analysis

## Step 4: Malformed Tag Detection and Repair

## Step 5: Non-HTML Tag Detection

## Step 6: Citation Extraction

## Step 7: Cross-Reference Extraction

## Step 8: Complete Workflow Example

## Individual Processing Functions

## Usage Examples

## DIY Data Clean-Up